In [2]:
import pandas as pd
import numpy as np
import os
import pickle
from tqdm import tqdm

import pyaudio  
import wave  

import shutil

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from collections import Counter

import json

import soundfile as sf

In [64]:
path = 'data\\train_opus\\'

In [65]:
tweets = []
name_json = ['1hour.jsonl', '10hour.jsonl', '100hour.jsonl', '10min.jsonl', 'manifest.jsonl']

for line in open(path + name_json[-1], 'r'):
    tweets.append(json.loads(line))

In [66]:
tweets[0]

{'id': '77b380796d242cf5bc09cfb551cffecd',
 'audio_filepath': 'crowd/0/77b380796d242cf5bc09cfb551cffecd.opus',
 'text': 'алиби',
 'duration': 1.21}

# Ф-я преобразования цифр в строчные значения

In [3]:
def as_words(n):
    """Convert an integer n (+ve or -ve) to English words."""
    # lookups
    ones = ['ноль', 'один', 'два', 'три', 'четыре',
            'пять', 'шесть', 'семь', 'восемь', 'девять', 
            'десять', 'одиннадцать', 'двенадцать', 'тринадцать', 'четырнадцать',
            'пятнадцать', 'шестнадцать', 'семнадцать', 'восемнадцать', 'девятнадцать']
    tens = ['ноль', 'десять', 'двадцать', 'тридцать', 'сорок',
            'пятьдесят', 'шестьдесяст', 'семьдесят', 'восемьдесят', 'девяносто']
    # negative case
    if n < 0:
        return "минус {0}".format(as_words(abs(n)))
    # 1000+
    for order, word in [(10**12, "трилион"), (10**9, "биллион"),
                        (10**6, "миллион"), (10**3, "тысячи")]:
        if n >= order:
            return "{0} {1}{2}".format(as_words(n // order), word,
                                       " {0}".format(as_words(n % order))
                                       if n % order else "")
    # 100-999
    if n >= 100:
        if n / 300 > 1:
            return "триста {1}".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 300 == 1:
            return "триста"
        elif n / 200 >= 1:
            return "двести".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 200 == 1:
            return "двести"
        elif n / 100 > 1:
            return "сто {1}".format(as_words(n // 100), 
                                                as_words(n % 100))
        elif n / 100 == 1:
            return "сто"
        
        else:
            return "{0} сто".format(as_words(n // 100))
    # 0-99
    if n < 20:
        return ones[n]
    else:
        return "{0}{1}".format(tens[n // 10],
                               " {0}".format(as_words(n % 10)) 
                               if n % 10 else "") 

In [4]:
main_word = []

In [5]:
word = ['килина', 'ванна','кухня','туалет','коридор','детская','спальня', 'включить', 'выключить']
# word = ['включить', 'выключить', 'точка', 'привет', "мотодепо", 'пока', 'вверх', 'вниз', 'влево', "вправо"]

In [6]:
number = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,
         100,200, 300]

In [7]:
for i in number:
    num = as_words(i)
#     print(num, i)
#     print('*' * 20)
    main_word.append(num)
len(main_word)

31

In [8]:
for i in word:
    main_word.append(i)
len(main_word)    

40

## Поиск слов в датасете

In [9]:
np.array(main_word)

array(['ноль', 'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь',
       'восемь', 'девять', 'десять', 'одиннадцать', 'двенадцать',
       'тринадцать', 'четырнадцать', 'пятнадцать', 'шестнадцать',
       'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать',
       'тридцать', 'сорок', 'пятьдесят', 'шестьдесяст', 'семьдесят',
       'восемьдесят', 'девяносто', 'сто', 'двести', 'триста', 'килина',
       'ванна', 'кухня', 'туалет', 'коридор', 'детская', 'спальня',
       'включить', 'выключить'], dtype='<U12')

In [10]:
main_word_norm = [morph.parse(i)[0] for i in main_word]
main_word_norm = [i.normal_form for i in main_word_norm]

# word_norm = morph.parse(word)[0]
# word_norm = word_norm.normal_form

In [11]:
np.array(main_word_norm)

array(['ноль', 'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь',
       'восемь', 'девять', 'десять', 'одиннадцать', 'двенадцать',
       'тринадцать', 'четырнадцать', 'пятнадцать', 'шестнадцать',
       'семнадцать', 'восемнадцать', 'девятнадцать', 'двадцать',
       'тридцать', 'сорок', 'пятьдесят', 'шестьдесястый', 'семьдесят',
       'восемьдесят', 'девяносто', 'сто', 'двести', 'триста', 'килина',
       'ванна', 'кухня', 'туалет', 'коридор', 'детский', 'спальня',
       'включить', 'выключить'], dtype='<U13')

In [11]:
# 'manifest.jsonl'
tweets = []
res = {}
count = 1
for dict_ in tweets:
    text = dict_['text']
    if len(text.split()) == 1:
        word_norm = morph.parse(text)[0]
        word_norm = word_norm.normal_form
        if text in main_word_norm:
            print(f"{name} : {text}")
            if text in res:
                res[text] += 1
            else:
                res[text] = 1
print('manifest.jsonl')       
print(res)

manifest.jsonl
{}


In [27]:
with open("data\\train_opus\\crowd\\test.txt", "w") as file:
    file.write("tes test test")

In [29]:
path = 'data\\train_opus\\'

name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
count_one = 0
count_all = 0
for name in name_json:
    print(name)
    tweets = []
    for line in open(path + name, 'r'):
        tweets.append(json.loads(line))
        res = {}
    for dict_ in tweets:
        text = dict_['text']
        path_file = dict_['audio_filepath']
        path_file = path + path_file
        name = dict_['id']
        if len(text.strip().split()) == 1:
    #         print(f"{name} : {text} len == 1")
            word_norm = morph.parse(text)[0]
            word_norm = word_norm.normal_form
            if word_norm in main_word_norm:
    #                     print(f"{name} : {text}")
                count_all += 1
                if text in res:
                    res[text] += 1
                else:
                    res[text] = 1
                shutil.copy(path_file, 'result_data\\2')
                with open(f"result_data\\2\\{name}.txt", "w") as file:
                    file.write(f"{text}")
#                 shutil.copy(path_txt, 'result_all_data')
#     print(name)       
    print(res)
count_all

1hour.jsonl
{}
10hours.jsonl
{'выключи': 9, 'один': 1, 'восемьдесят': 1, 'детская': 1}
100hours.jsonl
{'выключи': 96, 'два': 1, 'включи': 1, 'один': 1, 'девяносто': 1, 'детский': 3, 'пятнадцать': 1, 'кухня': 1, 'пять': 1, 'детские': 1}
10min.jsonl
{}
manifest.jsonl
{'семь': 3, 'один': 6, 'кухня': 14, 'пятнадцать': 2, 'включи': 13, 'четыре': 4, 'три': 5, 'два': 3, 'пять': 3, 'детский': 68, 'детские': 3, 'семнадцать': 1, 'одна': 1, 'шесть': 1, 'девять': 1, 'одни': 1, 'двадцать': 1, 'включу': 1, 'тринадцать': 2, 'тридцать': 1, 'одиннадцать': 1, 'девятнадцать': 2, 'восемьдесят': 1, 'выключи': 933, 'детская': 12, 'килине': 1, 'девяносто': 1}


1204

In [30]:
os.path.exists("crowd/8/86b9b825fe0f2a6bba4f2765bdba1bc7.wav")

False

In [50]:
path = 'data\\'

name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
count_one = 0
count_all = 0
for name in name_json:
    print(name)
    tweets = []
    
    for line in open(path + name, 'r', encoding = 'utf-8'):
        tweets.append(json.loads(line))
        res = {}
#         print(line)
    for dict_ in tweets:
        one_twe = []
        text = dict_['text']
        audio_filepath = dict_['audio_filepath']
        audio_filepath = 'data//' + audio_filepath
        name = dict_['id']
#         print(audio_filepath, audio_filepath.split('/')[1])
        if audio_filepath.split('/')[1] == 0 and len(text.split()) == 1:
            print(text)
        if len(text.strip().split()) == 1:
            if os.path.exists(audio_filepath):
#                 print(1)
                word_norm = morph.parse(text)[0]
                word_norm = word_norm.normal_form
                if word_norm in main_word_norm:
#                     print(audio_filepath)
                    if text in res:
                        res[text] += 1
                    else:
                        res[text] = 1
                
                    shutil.copy(audio_filepath, 'result_data\\2')
                    with open(f"result_data\\2\\{name}.txt", "w") as file:
                        file.write(f"{word_norm}")
            
    print(res)

1hour.jsonl
{}
10hours.jsonl
{'выключи': 9, 'детская': 1}
100hours.jsonl
{'выключи': 96, 'два': 1, 'включи': 1}
10min.jsonl
{}
manifest.jsonl
{'четыре': 1, 'три': 3, 'семь': 1, 'два': 1, 'пять': 1, 'один': 1, 'выключи': 933, 'детская': 12, 'включи': 3, 'килине': 1}


In [26]:
name_json = ['1hour.jsonl', '10hours.jsonl', '100hours.jsonl', '10min.jsonl', 'manifest.jsonl']
tweets = []
res = {}
count = 1

for line in open(path + name_json[0], 'r'):
    tweets.append(json.loads(line))
print(tweets[0])

{'id': 'cadbe57fe680d0581348a172820af423', 'audio_filepath': 'crowd/8/cadbe57fe680d0581348a172820af423.opus', 'text': 'улица молодежная дом семнадцать в', 'duration': 4.3649375}
